In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfTransformer
import html
import re
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import torch

In [ ]:
!pip install datasets

In [ ]:
df = pd.read_csv('sample_data/labeled_data.csv')

In [ ]:
df.drop(["hate_speech","Unnamed: 0","count","offensive_language","neither"],axis=1,inplace=True)

In [ ]:
def process(message):
    nopunc = [char for char in message if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    clean = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

    return ' '.join(clean)

In [ ]:
nltk.download('stopwords')
df["tweet"] = df["tweet"].apply(process)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def clean_tweet(tweet):
    # Unescape HTML entities (e.g., &amp; -> &, &#128536; -> emoji)
    tweet = html.unescape(tweet)
    # Remove mentions (@username)
    tweet = re.sub(r'@\w+', '', tweet)
    # Remove hashtags (#hashtag)
    tweet = re.sub(r'#\w+', '', tweet)
    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)
    # Remove emojis and non-ASCII characters
    tweet = tweet.encode('ascii', 'ignore').decode('ascii')
    # Remove special characters, punctuation (except sentence enders), and digits
    tweet = re.sub(r'[^A-Za-z\s]', '', tweet)
    # Collapse multiple spaces into one
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet

In [ ]:
df['tweet'] = df['tweet'].apply(clean_tweet)

In [ ]:
porter = nltk.PorterStemmer()
def stem(message):
    stemmedArr = [porter.stem(term) for term in message.split(" ")]
    return ' '.join(stemmedArr)

In [ ]:
df["tweet"]=df["tweet"].apply(stem)

In [ ]:
df = df.rename(columns={'tweet': 'text','class':'label'})

In [ ]:
df.tail()

,label,text
24778,1,you muthafin lie lifeask pearl coreyemanuel ri...
24779,2,youv gone broke wrong heart babi drove redneck...
24780,1,young buck wanna eat dat nigguh like aint fuck...
24781,1,youu got wild bitch tellin lie
24782,2,ruffl ntac eileen dahlia beauti color combin p...


In [ ]:
#map_dic = {0:"hate_speech",1:"offensive_language",2:"neither"}
#df["label"]=df["label"].map(map_dic)

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/24783 [00:00<?, ? examples/s]

In [ ]:
split = tokenized_datasets.train_test_split(test_size=0.3)
train_dataset = split["train"]
eval_dataset = split["test"]

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

ValueError: mount failed

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: charmaqh (charmaqh-ensam-casablanca) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.395700
1000,0.321600
1500,0.305100
2000,0.309300
2500,0.272800
3000,0.255200
3500,0.254200
4000,0.251300
4500,0.240200
5000,0.208500


Step,Training Loss
500,0.395700
1000,0.321600
1500,0.305100
2000,0.309300
2500,0.272800
3000,0.255200
3500,0.254200
4000,0.251300
4500,0.240200
5000,0.208500


In [ ]:
trainer.evaluate()

{'eval_loss': 0.3521217405796051,
 'eval_runtime': 210.4282,
 'eval_samples_per_second': 35.333,
 'eval_steps_per_second': 4.42,
 'epoch': 3.0}

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

In [ ]:
trainer.compute_metrics = compute_metrics
results = trainer.evaluate()
print(results)

{'eval_loss': 0.3521217405796051, 'eval_accuracy': 0.908675184936113, 'eval_runtime': 211.7833, 'eval_samples_per_second': 35.107, 'eval_steps_per_second': 4.391, 'epoch': 3.0}


In [ ]:
trainer.save_model("fine_tuned_bert_model")